In [2]:
  
import requests
import zlib
import json
import base64
import time
import copy
import random
from cqhttp import CQHttp
from apscheduler.schedulers.blocking import BlockingScheduler
import pandas as pd


def addSalt(ori: bytearray):
    # 从网页JS当中提取到的混淆盐值，每隔一位做一次异或运算
    Salt = '%#54$^%&SDF^A*52#@7'
    i = 0
    for ch in ori:
        if i % 2 == 0:
            ch = ch ^ ord(Salt[(i // 2) % len(Salt)])
        ori[i] = ch
        i += 1
    return ori


def encodeData(ori: str):
    # 开头的数字是原始报文长度
    Length = len(ori)
    Message = str.encode(ori)
    # 首先用zlib进行压缩
    Compressed = bytearray(zlib.compress(Message))
    # 然后加盐混淆
    Salted = addSalt(Compressed)
    # 最后将结果转化为base64编码
    Result = base64.b64encode(Salted).decode('utf-8')
    # 将长度头和base64编码的报文组合起来
    return str(Length) + '$' + Result


def decodeData(ori: str):
    # 分离报文长度头
    # tbc: 增加报文头长度的验证
    Source = ori.split('$')[1]
    # base64解码
    B64back = bytearray(base64.b64decode(Source))
    # 重新进行加盐计算，恢复原始结果
    Decompressed = addSalt(B64back)
    # zlib解压
    Result = zlib.decompress(Decompressed).decode('utf-8')
    # 提取json
    return json.loads(Result)


header = {
    'Content-Type': 'application/json',
    'Origin': 'https://www.tao-ba.club',
    'Accept-Language': 'zh-CN,zh;q=0.9',
    'Host': 'www.tao-ba.club',
    'Cookie': 'l10n=zh-cn',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0.5 Safari/605.1.15',
    'Referer': 'https://www.tao-ba.club/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
}
cookie = {
    'l10n': 'zh-cn',
}


# 读取配置文件
def config():
    with open("config.json", 'r', encoding='utf-8') as f:
        conf = json.load(f)
    proj_id = conf['project_id']
    page = conf['page']
    nick = conf['nick']
    text = conf['text']
    target = conf['target']
    return proj_id, page, nick, text, target


def getInfo(id):
    url = "https://www.taoba.club/idols/join/rank"
    #url = "https://www.taoba.club/idols/sitelevel/stats"
    #url = "https://www.taoba.club/idols/dtask/rank"
    pkgroup = "210406knu9OwEO"

    data = '{{"id": "{}", "pkgroup":"{}","requestTime": {},"ismore":"false","limit":15,"offset": 0,"minid":10582663,"pf": "h5"}}'.format(id,pkgroup, int(time.time() * 1000))
    sid = 3707
    #data = '{{"sid":"{}","pkgroup":"{}",requestTime":{},"pf":"h5"}}'.format(sid,pkgroup,int(time.time()*1000))
   
    #data = '{{"pkgroup":"{}","requestTime":{},"pf":"h5"}}'.format("210403dmrC1AjI",int(time.time()*1000))
    data = encodeData(data)
    response = requests.post(url=url, headers=header, data=data, cookies=cookie)
    response = decodeData(response.text)
    ##cur_money = response['datas']['donation']
    #t = float(cur_money / target * 100)
    #res = response['datas']['title'] + " 目前已筹：¥" + str(cur_money) + "，目标金额：¥" + str(target) + "集资进度：" + str(
        #'%.2f'% t) + "%\n"
    #res += "集资链接：https://www.tao-ba.club/#/pages/idols/detail?id=" + str(id)
    #return res, cur_money
    url2 = "https://www.taoba.club/idols/detail"
    
    response2 = requests.post(url = url2,headers=header,data=data,cookies=cookie)
    
    response2 = decodeData(response2.text)
    star = response2["datas"]["star"]
    cur_money = response2["datas"]["donation"]
    
    
    return star,cur_money,response["list"]

In [3]:

def getStats(jizi_id,ppl=500):
    star,donation,dic = getInfo(jizi_id)
    donation_list = []
    for i in dic:
        donation_list.append(i["money"])
    top51_percentage = (sum(donation_list)/donation) * 100
    greater_than_1w = 0
    for i in donation_list:
        if i>=10000:
            greater_than_1w +=i
    greater_than_1w_percentage = (greater_than_1w/donation)*100
    average = donation/ppl
    
    return_dic={}

    return_dic["训练生"] = star
    return_dic["总额(￥)"] = f"{donation:,}"
    return_dic["total"] = donation
    return_dic["人头数"] = ppl
    return_dic["人均(￥)"] = round(average,2)
    return_dic["榜单第五十一名集资金额(￥)"] = min(donation_list)
    return_dic["大额占比(>1w)"] = str(round(greater_than_1w_percentage,2)) + "%"
    return return_dic


In [4]:
lgy_previous = 207401
xxc_previous = 251170
hdr_previous = 186486
from datetime import datetime
import pytz
  
# get the standard UTC time 
UTC = pytz.utc
  
# it will get the time zone 
# of the specified location
IST = pytz.timezone('Asia/Shanghai')
  
# print the date and time in
# standard format
print("UTC in Default Format : ", 
      datetime.now(UTC))
  
print("IST in Default Format : ", 
      datetime.now(IST))
  
# print the date and time in 
# specified format
datetime_utc = datetime.now(UTC)
print("Date & Time in UTC : ",
      datetime_utc.strftime('%Y:%m:%d %H:%M:%S %Z %z'))
  
datetime_ist = datetime.now(IST)
print("Date & Time in IST : ", 
      datetime_ist.strftime('%Y:%m:%d %H:%M:%S %Z %z'))
current_time = datetime.now(IST)
last_time = current_time

UTC in Default Format :  2021-04-24 05:01:52.589820+00:00
IST in Default Format :  2021-04-24 13:01:52.590054+08:00
Date & Time in UTC :  2021:04:24 05:01:52 UTC +0000
Date & Time in IST :  2021:04:24 13:01:52 CST +0800


In [14]:
hdr = getStats(17735,2118)
lgy = getStats(17710,2466)
xxc = getStats(17729,3375)
last_time_formatted = last_time.strftime('%H:%M')+"（上个时间点）总额(￥)"
current_time = datetime.now(IST)
current_time_formatted = current_time.strftime('%H:%M')

time_lapsed = int(round((current_time-last_time).total_seconds()/60,0))
last_time = current_time
time_lapsed = str(time_lapsed)+"分钟涨幅(￥)"
hdr[last_time_formatted] = hdr_previous
lgy[last_time_formatted] = lgy_previous
xxc[last_time_formatted] = xxc_previous


hdr[time_lapsed] = hdr["total"] - hdr_previous
lgy[time_lapsed] = lgy["total"] - lgy_previous
xxc[time_lapsed] = xxc["total"] - xxc_previous

def getDf(li):
    df_li = []
    for dic in li:
        dic_df = pd.DataFrame.from_dict(dic,orient="index")
        df_li.append(dic_df)
    df = pd.concat(df_li,axis=1)
    df.columns = df.iloc[0]
    return df.drop(df.index[0])

print("卡位圈4/23 团建数据，当前时间为 "+current_time_formatted)

df = getDf([hdr,lgy,xxc])
display(df)


hdr_previous = hdr["total"]
lgy_previous = lgy["total"]
xxc_previous = xxc["total"]

print("和lgy家差距"+str(round(lgy_previous - hdr_previous,1)))
print("和xxc家差距"+str(round(xxc_previous-hdr_previous,1)))

卡位圈4/23 团建数据，当前时间为 14:15


训练生,何德瑞,刘冠佑,徐新驰
总额(￥),"226,420.4","308,502.36","257,964"
total,226420,308502,257964
人头数,2118,2466,3375
人均(￥),106.9,125.1,76.43
榜单第五十一名集资金额(￥),1111,917.12,612
大额占比(>1w),4.46%,9.08%,28.44%
14:09（上个时间点）总额(￥),225646,306756,257959
6分钟涨幅(￥),774.6,1746.2,5


和lgy家差距82082.0
和xxc家差距31543.6


In [1]:
import matplotlib.pyplot as plt
import pandas as pd
from pandas.table.plotting import table # EDIT: see deprecation warnings below

ax = plt.subplot(111, frame_on=False) # no visible frame
ax.xaxis.set_visible(False)  # hide the x axis
ax.yaxis.set_visible(False)  # hide the y axis

table(ax, df)  # where df is your data frame



ModuleNotFoundError: No module named 'pandas.table'

UTC in Default Format :  2021-04-23 15:42:09.847334+00:00
IST in Default Format :  2021-04-23 23:42:09.847675+08:00
Date & Time in UTC :  2021:04:23 15:42:09 UTC +0000
Date & Time in IST :  2021:04:23 23:42:09 CST +0800


In [ ]:
def getList(id, page):
    url = "https://www.tao-ba.club/idols/detail"
    base = '{{"starid":3943,"pkgroup":"210403dmrC1AjI","ismore":{},"limit":{},"id":"{}","offset":{},"requestTime":{},"pf":"h5"}}'
    flag = False
    offset = 0
    global dic
    while True:
        if flag:
            f = "true"
        else:
            f = "false"
        data = encodeData(base.format(f, page, id, offset, int(time.time() * 1000)))
        print(decodeData(requests.post(url=url, headers=header, data=data).text))
        response = decodeData(requests.post(url=url, headers=header, data=data).text)["list"]
        for x in response:
            dic[x['userid']] = [x['id'], float(x['money']), x['nick'], x['stime']]
        flag = len(response) == 15
        offset += page
        if flag == False:
            break
    return dic

In [6]:
import requests
import lxml
from bs4 import BeautifulSoup
import pandas as pd
import plotly.express as px
overall_data = {
    "心引力":{"cid":"305609583","people":{"1":"何德瑞","2":["唐九洲","jojo"],"3":"王梓澳","4":"李添翼","5":"邱丹枫","6":"周峻宇"}},
}

people_dict = {"1":"何德瑞","2":["余景天","tony"],"3":"孙滢皓","4":"十七","5":"亿轩"}
def get_graph(people_dict,cid,group_name):
    url = "http://comment.bilibili.com/cid.xml".replace("cid",str(cid))
    html = requests.get(url).content
    html_data = str(html,"utf-8")
    soup = BeautifulSoup(html_data,"lxml")
    results = soup.find_all("d")
    comments = [comment.text for comment in results]
    danmu_df = pd.DataFrame(comments)
    danmu_df.columns=["comment"]
    danmu_dict = {}
    for key in people_dict.keys():
        name = people_dict[key]
        if isinstance(name,str):
            danmu_dict[name] = danmu_df.comment.str.contains(name).sum()
        if isinstance(name,list):
            count = 0
            for alt_name in name:
                count += danmu_df.comment.str.contains(alt_name).sum()
            danmu_dict["-".join(name)] = count
    df = pd.DataFrame.from_dict(danmu_dict,orient="index",columns = ["最新1000条弹幕含量"]).reset_index().rename({"index":"爱豆名字"},axis=1)
    fig = px.bar(df, x='爱豆名字', y='最新1000条弹幕含量', text='最新1000条弹幕含量',title=group_name+"组最新1000条弹幕构成")
    fig.show()
    return danmu_df
    
people_dict =  {"1":"何德瑞","2":"常华森","3":"喻言","4":"陆柯燃"}
danmu_df = get_graph(people_dict,"331065947","Yes Ok")

In [3]:
cid = 316569405
show_name = "Okay B"

people_dict = {"1":"梁森","2":"王浩轩","3":"万禹辰","4":"陈俊豪","5":"艾克里里"}

danmu_df = get_graph(people_dict,cid,show_name)

In [4]:
cid = 316576353
show_name = "Stop Sugar A"

people_dict = {"1":"周子杰","2":"李俊濠","3":"杨昊铭","4":"许卓伦","5":"苛尔力钧"}

danmu_df = get_graph(people_dict,cid,show_name)

In [ ]:
cid = 316576353
show_name = "Stop Sugar B"

people_dict = {"1":"周子杰","2":"李俊濠","3":"杨昊铭","4":"许卓伦","5":"苛尔力钧"}

danmu_df = get_graph(people_dict,cid,show_name)